In [ ]:
# Some useful packages
import numpy as np         # linear algebra
import sklearn as sk       # machine learning
import pandas as pd        # reading in data files, data cleaning
import matplotlib.pyplot as plt   # for plotting
import seaborn as sns      # visualization tool
import tensorflow as tf
import keras

In [ ]:
from keras.models import Sequential  
from keras.layers import Dense  
from keras.layers import LSTM  
from keras.layers import Dropout  

In [ ]:
# Random Forest
# ARIMA
# LSTM

In [ ]:
data = pd.read_csv('full.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
data.head()

In [ ]:
#df[0:4110].to_csv('train.csv')

In [ ]:
#df[4110:5137].to_csv('test.csv')

In [ ]:
#5137 -> 
# 80% = 4110 samples
# 20% = 1027 samples

# Feature Modifications

In [ ]:
data['Dew Point'] = data['Dew Point'].str.replace('F', '')
data['Temperature'] = data['Temperature'].str.replace('F', '')
data['Humidity'] = data['Humidity'].str.replace('%', '')
data['Wind Speed'] = data['Wind Speed'].str.replace('mph', '')
data['Pressure'] = data['Pressure'].str.replace('in', '')
data['Wind Gust'] = data['Wind Gust'].str.replace('mph', '')


In [ ]:
data.set_index('Datetime',inplace=True)

In [ ]:
cols = ['Temperature', 'Dew Point', 'Humidity','Wind Speed','Wind Gust','Pressure']
data[cols] = data[cols].apply(pd.to_numeric)


In [ ]:
data.head()

In [ ]:
#train_X = train[['Weekday','Part of the day', 
 #                'Temperature', 'Humidity',
  #               'Wind Speed', 'Wind Gust', 'State Holiday']]
#input dim = 7 

#train_y = train['counts']

In [ ]:
#use temporarily because of issue with reshaping
#train_X = train[[ 'Temperature', 'Humidity',
 #                'Wind Speed', 'Wind Gust', 'State Holiday']]
#input dim = 5

#train_y = train['counts']

#test_X = test[[
 #                'Temperature', 'Humidity',
  #               'Wind Speed', 'Wind Gust', 'State Holiday']]
#test_Y = test['counts']

# Single Feature Rolling Forecast ARIMA

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
df = data[['counts']]

In [ ]:
df.head()

In [ ]:
df.plot()

In [ ]:
X = df.values
size = int(len(X) *0.66) #66/34 train/split

In [ ]:
train, test = X[0:size], X[size:len(X)]


In [ ]:
history = [x for x in train]
predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0].round()
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

In [ ]:
from sklearn.metrics import mean_squared_error
error = mean_squared_error(test, predictions)

from math import sqrt
rms = sqrt(error) #root mean squared error

In [ ]:
from sklearn.metrics import mean_squared_error
error = mean_squared_error(test, predictions)
from math import sqrt
rms = sqrt(error) #root mean squared error

In [ ]:
print('Test MSE: %.3f' % error)
print('Test RMSE: %.3f' % rms)

In [ ]:
plt.plot(test)
plt.plot(predictions, color='red')
plt.xlim(200, 400)
plt.show()

# Random Forest - Multivariate

In [ ]:
# https://pythondata.com/forecasting-with-random-forests/

In [68]:
X = df.values
size = int(len(X) *0.66) #66/34 train/split

In [69]:
train, test = X[0:size], X[size:len(X)]

In [70]:
from sklearn.ensemble.forest import RandomForestRegressor
# build our RF model
RF_Model = RandomForestRegressor(n_estimators=100,
                                 max_features=1, oob_score=True)

In [ ]:
# let's get the labels and features in order to run our 
# model fitting
labels = y_train #[:, None]
features = X_train[:, None]

# Fit the RF model with features and labels.
rgr=RF_Model.fit(features, labels)

# Now that we've run our models and fit it, let's create
# dataframes to look at the results
X_test_predict=pd.DataFrame(
    rgr.predict(X_test[:, None])).rename(
    columns={0:'predicted_price'}).set_index('predicted_price')
X_train_predict=pd.DataFrame(
    rgr.predict(X_train[:, None])).rename(
    columns={0:'predicted_price'}).set_index('predicted_price')

# combine the training and testing dataframes to visualize
# and compare.
RF_predict = X_train_predict.append(X_test_predict)

# LSTM

In [ ]:

#Rakshit's model
    
inp = kl.Input((None,))
x = kl.LSTM(128, return_sequences=True)(x)
x = kl.GlobalMaxPool1D()(x)
x = kl.Dense(64, activation='sigmoid')(x)
out = kl.Dense(n_classes, activation='sigmoid')(x)

model = keras.Model(inputs=inp, outputs=out)
model.summary()


In [ ]:
# Resizing
#X, y = np.array(train_X), np.array(train_y)
X = train_X.to_numpy()
print(X.shape)
#X = X.reshape(X.shape[0],50, 7)
#X = X.reshape(X.shape[0], 50, 7)
#print(X.shape)
#print(X_test.shape)

In [ ]:
print(X[0])

In [ ]:
# reshape input to be 3D [samples, timesteps, features]
temp_x = X.reshape((4110, 50, 5))
print(train_X.shape)

In [ ]:
# https://machinelearningmastery.com/prepare-univariate-time-series-data-long-short-term-memory-networks/
samples = list()
length = 137
n = 4110
for i in range(0,n,length):
    sample= train[i:i+length]
    samples.append(sample)
#data = array(data)
#print(data[:5, :])
print(len(samples))

In [ ]:
d = np.matrix(samples)
print(d.shape)

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
            # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 

In [ ]:
full = pd.read_csv('full.csv')

In [ ]:
full.info()

In [ ]:
full.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
full.set_index('Datetime',inplace=True)

In [ ]:
full.head()

In [ ]:
full['Dew Point'] = full['Dew Point'].str.replace('F', '')
full['Temperature'] = full['Temperature'].str.replace('F', '')
full['Humidity'] = full['Humidity'].str.replace('%', '')
full['Wind Speed'] = full['Wind Speed'].str.replace('mph', '')
full['Pressure'] = full['Pressure'].str.replace('in', '')
full['Wind Gust'] = full['Wind Gust'].str.replace('mph', '')


In [ ]:
full.head()

In [ ]:
#use temporarily because of issue with reshaping
df = full[['Temperature', 'Humidity','Wind Speed', 'Wind Gust', 'State Holiday','counts']]
#input dim = 5

#train_y = train['counts']

#test_X = full[[
 #                'Temperature', 'Humidity',
  #               'Wind Speed', 'Wind Gust', 'State Holiday','counts']]
#test_Y = test['counts']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

values = df.values
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled


In [ ]:

reframed = series_to_supervised(scaled,1,1)
print(reframed.head())

In [ ]:
values = reframed.values
n_train_hours = 82*50
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

#split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

#reshape to 3d
train_X = train_X.reshape((train_X.shape[0],1,train_X.shape[1]))
test_X =test_X.reshape((test_X.shape[0],1,test_X.shape[1]))
print(train_X.shape,train_y.shape, test_X.shape, test_y.shape)

In [ ]:
print("X_train size", train_X.shape)
print("y_train size", train_y.shape)

print("X_test size", test_X.shape)
print("y_test size", test_y.shape)


In [ ]:
# design network
model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error','accuracy'])
# fit network
#history = model.fit(train_X, train_y, epochs=100, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
history = model.fit(train_X, train_y, epochs=100, batch_size=72, verbose=2, shuffle=False)

# plot history
plt.plot(history.history['loss'], label='train')
#plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

#accuracy


In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error

In [ ]:
yhat = model.predict(test_X)
 
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
#print(test_X.shape)



In [ ]:
# invert scaling for prediction
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]



In [ ]:
print(inv_yhat.shape)
print(inv_y.shape)

In [ ]:
# calculate RMSE
rmse = mean_squared_error(test_y, yhat)
print('Test RMSE', rmse)

In [ ]:
#plot yhat and testy
plt.plot(yhat,color='red', label='predicted')
plt.plot(test_y,color='blue',label='actual')
plt.title('Results')
plt.xlabel('Time')    f
plt.ylabel('Accident Counts')
plt.legend()
plt.xlim(0,200)

plt.show()

In [ ]:
train_loss, train_acc = model.evaluate(test_X,test_y)

In [ ]:
print('Training set accuracy:', train_acc)


# Attempt 2

In [ ]:
#use temporarily because of issue with reshaping
data = full[['Temperature', 'Humidity',
                 'Wind Speed', 'Wind Gust', 'State Holiday']]
#input dim = 5

#train_y = train['counts']

#test_X = test[[
              #   'Temperature', 'Humidity',
               #  'Wind Speed', 'Wind Gust', 'State Holiday']]
#test_Y = test['counts']

In [ ]:
full.head()

In [ ]:
# keep datetime feature as a column
# sin(2pih/24)

In [ ]:
# StandardScaler instead of minmax because minmax is prone to deviation

In [ ]:
#use full.csv
values = data.values
values = values.astype('float32')
values

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
#MinMaxScaler
scaled_data = scaler.fit_transform(values)

In [ ]:
scaled_data.shape

In [ ]:
y_temp = full['counts'].values
y_temp.shape

In [ ]:
y = np.array(y_temp)

In [ ]:
#y = y_temp.reshape((-1, 1))

In [ ]:
y.shape

In [ ]:
def window_data(data, window_size):
    X = []
    
    i = 0
    while (i+ window_size) < len(data) :
        X.append(data[i:i+window_size])
        i+=1
    return X


In [ ]:
X = window_data(scaled_data, 14)

In [ ]:
len(X)

In [ ]:
#window size
import numpy as np
X_train = np.array(X[:4110])
y_train = np.array(y[:4110])

X_test = np.array(X[4110:])
y_test = np.array(y[4110:5123])


In [ ]:
print("X_train size", X_train.shape)
print("y_train size", y_train.shape)

print("X_test size", X_test.shape)
print("y_test size", y_test.shape)


In [ ]:
print(X_test.shape[0])

In [ ]:
# define variables
batch_size = 14
window_size = 14


In [ ]:
# design network
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_test.shape[2])))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.summary()
# fit network
history = model.fit(X_train, y_train, epochs=100, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False)
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

#accuracy


In [ ]:
 
#print(test_X.shape)





# Attempt 3

In [ ]:
data = full[['Temperature', 'Humidity',
                 'Wind Speed', 'Wind Gust', 'State Holiday']]
#counts

In [ ]:
full.head()

In [ ]:
input_features = data.values
input_data = input_features

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
input_data = sc.fit_transform(input_data)

In [ ]:
input_data.shape

In [ ]:
def window_data(data, window_size):
    X = []
    y = []
    
    i = 0
    while (i+ window_size) <= len(data) -1:
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
        i+=1
    assert len(X) == len(y)
    return X, y

In [ ]:
scaled

In [ ]:
lookback = 14
test_size = int(.2 * len(data))

X=[]
y=[]

for i in range(len(data)-lookback-1):
    t =[]
    for j in range(0,lookback):
        t.append(input_data[[(i+j)],:])
    X.append(t)
    y.append(input_data[i+lookback,1])

In [ ]:
X,y = np.array(X) , np.array(y)
X_test =  X[:test_size+lookback]

feature_size = 5

X = X.reshape(X.shape[0],lookback,feature_size)
X_test = X_test.reshape(X_test.shape[0],lookback, feature_size)

In [ ]:
print(X.shape)
print(X_test.shape)

In [ ]:
from keras import Sequential
from keras.layers import Dense, LSTM

In [ ]:
model = Sequential()
model.add(LSTM(30,return_sequences=True, input_shape=(X.shape[1],feature_size)))
model.add(LSTM(30,return_sequences=True))
model.add(LSTM(30))
model.add(Dense(1))
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
model.fit(X,y,epochs=100,batch_size=2)


# GRU

In [ ]:
inp = kl.Input((None,))
x = kl.GRU(128, return_sequences=True)(x)
x = kl.GlobalMaxPool1D()(x)
x = kl.Dense(64, activation='sigmoid')(x)
out = kl.Dense(n_classes, activation='sigmoid')(x)
    
model = keras.Model(inputs=inp, outputs=out)
model.summary()

In [ ]:
# try linear regression, try ARIMA, try random forest
# mpe for metrics?

# Evaluation

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics ='')


In [ ]:
model.fit(train_X, train_y, epochs=200, batch_size=32)
